# **Modelos Baseados em Árvores**
Os modelos baseados em árvores são uma classe poderosa de algoritmos de aprendizado de máquina utilizados tanto para tarefas de classificação quanto de regressão. Eles funcionam construindo uma estrutura hierárquica (árvore) de decisões baseadas nas features dos dados, permitindo capturar relações não lineares e interações complexas entre as variáveis.

**Características dos Modelos Baseados em Árvores**
- **Flexibilidade:** Capazes de modelar relações não lineares e interações entre features.
- **Interpretabilidade:** Árvores de decisão são intuitivas e fáceis de visualizar.
- **Robustez:** Modelos como Random Forest e Gradient Boosting são robustos a overfitting quando bem ajustados.
- **Escalabilidade:** Podem ser eficientes em conjuntos de dados de grande dimensão com as devidas otimizações.

No entanto, os modelos baseados em árvores também apresentam limitações, como a tendência a `overfitting` (especialmente árvores individuais profundas), e a necessidade de ajuste cuidadoso dos `hiperparâmetros` para alcançar um desempenho ótimo.

In [1]:
# Importando bibliotecas necessárias
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Importando modelos baseados em árvores do scikit-learn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier

# Importando XGBoost e LightGBM
try:
    import xgboost as xgb
except ImportError:
    !pip install xgboost
    import xgboost as xgb

try:
    import lightgbm as lgb
except ImportError:
    !pip install lightgbm
    import lightgbm as lgb

# Carregando o dataset breast_cancer
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

# Dividindo o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Função para treinar, prever e avaliar modelos
def train_evaluate_model(model, model_name, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"### {model_name}")
    print("Acurácia:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))



/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


##**Decision Tree (Árvore de Decisão)**
**Funcionamento**
A Árvore de Decisão é o modelo base para muitos algoritmos de árvore. Ela divide recursivamente o espaço de features em regiões mais homogêneas em termos da variável alvo. Em cada nó interno, uma feature e um limiar são selecionados para melhor separar as classes (no caso de classificação) ou minimizar o erro (no caso de regressão).

**Pontos Fortes**
- Interpretabilidade: Fácil de visualizar e entender as decisões tomadas.
- Não Requer Pré-processamento: Não necessita de normalização ou padronização das features.
- Capacidade de Capturar Relações Não Lineares: Pode modelar interações complexas entre as features.

**Pontos Fracos**
- Tendência a Overfitting: Árvores profundas podem se ajustar excessivamente aos dados de treinamento.
- Instabilidade: Pequenas variações nos dados podem resultar em árvores muito diferentes.
- Baixo Desempenho em Dados Complexos: Árvores únicas podem não capturar todas as nuances dos dados.

In [2]:
# 1. Decision Tree
dt_clf = DecisionTreeClassifier(random_state=42)
train_evaluate_model(dt_clf, "Decision Tree", X_train, X_test, y_train, y_test)

### Decision Tree
Acurácia: 0.9415204678362573
              precision    recall  f1-score   support

           0       0.90      0.95      0.92        63
           1       0.97      0.94      0.95       108

    accuracy                           0.94       171
   macro avg       0.93      0.94      0.94       171
weighted avg       0.94      0.94      0.94       171



# **Random Forest**
**Funcionamento:**
O Random Forest é um ensemble de Árvores de Decisão. Ele constrói múltiplas árvores durante o treinamento e combina suas previsões por votação (classificação) ou média (regressão). Cada árvore é treinada em um subconjunto aleatório dos dados e das features, promovendo a diversidade entre as árvores.

**Pontos Fortes**
- Redução de Overfitting: A média de múltiplas árvores reduz a variância e melhora a generalização.
- Importância das Features: Fornece métricas de importância das features.
- Robustez: Funciona bem com dados ruidosos e de alta dimensionalidade.

**Pontos Fracos**
- Menos Interpretável: Mais difícil de visualizar e interpretar comparado a uma única árvore.
- Maior Custo Computacional: Requer mais memória e tempo de processamento devido à construção de múltiplas árvores.
- Não Funciona Bem com Dados de Sequência: Pode ter desempenho limitado em dados com dependências temporais.

In [3]:
# 2. Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
train_evaluate_model(rf_clf, "Random Forest", X_train, X_test, y_train, y_test)



### Random Forest
Acurácia: 0.9707602339181286
              precision    recall  f1-score   support

           0       0.98      0.94      0.96        63
           1       0.96      0.99      0.98       108

    accuracy                           0.97       171
   macro avg       0.97      0.96      0.97       171
weighted avg       0.97      0.97      0.97       171



##**Extra Trees (Extremely Randomized Trees)**
**Funcionamento:**
O Extra Trees é semelhante ao Random Forest, mas com uma maior aleatoriedade na construção das árvores. Ele seleciona os cortes das features de forma completamente aleatória, ao invés de buscar o melhor corte possível. Isso pode levar a uma redução do overfitting e a uma melhoria na velocidade de treinamento.

**Pontos Fortes**
- Rapidez: Treinamento mais rápido comparado ao Random Forest devido à aleatoriedade adicional.
- Redução de Overfitting: Aumenta a diversidade das árvores, melhorando a generalização.
- Bom Desempenho em Geral: Frequentemente apresenta desempenho similar ou superior ao Random Forest.

**Pontos Fracos**
- Menos Precisão em Algumas Situações: A aleatoriedade extra pode, em alguns casos, levar a uma ligeira queda na precisão.
- Interpretabilidade Reduzida: Semelhante ao Random Forest, é difícil de interpretar individualmente.
- Dependência de Hiperparâmetros: Requer ajuste cuidadoso dos hiperparâmetros para maximizar o desempenho.

In [4]:
# 3. Extra Trees
et_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
train_evaluate_model(et_clf, "Extra Trees", X_train, X_test, y_train, y_test)

### Extra Trees
Acurácia: 0.9766081871345029
              precision    recall  f1-score   support

           0       0.98      0.95      0.97        63
           1       0.97      0.99      0.98       108

    accuracy                           0.98       171
   macro avg       0.98      0.97      0.97       171
weighted avg       0.98      0.98      0.98       171



##**Gradient Boosting (Boosting por Gradiente)**
**Funcionamento:**
O Gradient Boosting constrói um modelo preditivo de forma sequencial, onde cada novo modelo tenta corrigir os erros dos modelos anteriores. Utiliza o gradiente da função de perda para direcionar a construção das novas árvores, otimizando a performance do conjunto.

**Pontos Fortes**
- Alto Desempenho: Geralmente alcança resultados de ponta em várias competições de machine learning.
- Flexibilidade: Pode otimizar diferentes funções de perda e ser adaptado para várias tarefas.
- Importância das Features: Fornece métricas de importância das features.

**Pontos Fracos**
- Treinamento Lento: Processo sequencial pode ser computacionalmente intensivo.
- Sensível a Outliers: Pode ajustar excessivamente os outliers nos dados.
- Requer Ajuste Cuidadoso: Muitos hiperparâmetros a serem ajustados para evitar overfitting.

In [5]:
# 4. Gradient Boosting
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
train_evaluate_model(gb_clf, "Gradient Boosting", X_train, X_test, y_train, y_test)

### Gradient Boosting
Acurácia: 0.9590643274853801
              precision    recall  f1-score   support

           0       0.95      0.94      0.94        63
           1       0.96      0.97      0.97       108

    accuracy                           0.96       171
   macro avg       0.96      0.95      0.96       171
weighted avg       0.96      0.96      0.96       171



##**AdaBoost (Adaptive Boosting)**
**Funcionamento:**
O AdaBoost é um algoritmo de boosting que combina múltiplos classificadores fracos (geralmente árvores de decisão rasas) em um classificador forte. Ele ajusta os pesos das amostras após cada iteração, dando mais importância às amostras que foram classificadas incorretamente.

**Pontos Fortes**
- Simplicidade: Fácil de implementar e entender.
- Redução de Erro: Pode melhorar significativamente o desempenho de classificadores fracos.
- Robustez a Overfitting: Menos propenso a overfitting em comparação com alguns outros métodos de boosting.

**Pontos Fracos**
- Sensível a Outliers: Outliers podem receber altos pesos, afetando negativamente o desempenho.
- Dependência dos Classificadores Fracos: Desempenho limitado pela qualidade dos classificadores base.
- Menor Precisão em Comparação com Boosting Avançado: Pode não alcançar a mesma precisão que métodos como Gradient Boosting ou XGBoost.

In [6]:
# 5. AdaBoost
ada_clf = AdaBoostClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
train_evaluate_model(ada_clf, "AdaBoost", X_train, X_test, y_train, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


### AdaBoost
Acurácia: 0.9532163742690059
              precision    recall  f1-score   support

           0       0.94      0.94      0.94        63
           1       0.96      0.96      0.96       108

    accuracy                           0.95       171
   macro avg       0.95      0.95      0.95       171
weighted avg       0.95      0.95      0.95       171



##**Bagging (Bootstrap Aggregating)**
**Funcionamento:**
O Bagging é uma técnica de ensemble que constrói múltiplos modelos treinados em diferentes subconjuntos do conjunto de dados original (obtidos por amostragem com reposição). As previsões dos modelos são então agregadas por votação ou média para formar a previsão final.

**Pontos Fortes**
- Redução de Overfitting: A média ou votação de múltiplos modelos reduz a variância.
- Versatilidade: Pode ser aplicado a vários tipos de modelos base.
- Melhoria de Desempenho: Geralmente melhora a precisão e a robustez dos modelos individuais.

**Pontos Fracos**
- Maior Custo Computacional: Requer treinamento de múltiplos modelos, aumentando o tempo de processamento.
- Menos Interpretável: Difícil de interpretar o modelo final como um todo.
- Benefícios Limitados para Alguns Modelos: Pode não melhorar significativamente modelos que já são robustos.

In [7]:
# 6. Bagging
bag_clf = BaggingClassifier(n_estimators=100, random_state=42)
train_evaluate_model(bag_clf, "Bagging", X_train, X_test, y_train, y_test)

### Bagging
Acurácia: 0.9590643274853801
              precision    recall  f1-score   support

           0       0.95      0.94      0.94        63
           1       0.96      0.97      0.97       108

    accuracy                           0.96       171
   macro avg       0.96      0.95      0.96       171
weighted avg       0.96      0.96      0.96       171



##**XGBoost (Extreme Gradient Boosting)**
**Funcionamento**
O XGBoost é uma implementação otimizada de Gradient Boosting que utiliza técnicas avançadas como regularização, paralelização e processamento distribuído. É projetado para ser altamente eficiente, flexível e portátil, oferecendo alto desempenho em diversas tarefas de machine learning.

**Pontos Fortes**
- Alto Desempenho: Frequentemente vence competições de machine learning devido à sua eficiência e capacidade de otimização.
- Regularização: Inclui termos de regularização que ajudam a prevenir overfitting.
- Flexibilidade: Suporta diferentes tipos de funções de perda e personalizações.

**Pontos Fracos**
- Complexidade: Mais complexo de configurar e otimizar comparado a outros algoritmos.
- Requer Conhecimento Avançado: Para tirar o máximo proveito, é necessário entender profundamente os hiperparâmetros e o funcionamento interno.
- Consumo de Memória: Pode ser intensivo em memória para grandes conjuntos de dados.

In [8]:
# 7. XGBoost
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
train_evaluate_model(xgb_clf, "XGBoost", X_train, X_test, y_train, y_test)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:32:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


### XGBoost
Acurácia: 0.9707602339181286
              precision    recall  f1-score   support

           0       0.95      0.97      0.96        63
           1       0.98      0.97      0.98       108

    accuracy                           0.97       171
   macro avg       0.97      0.97      0.97       171
weighted avg       0.97      0.97      0.97       171



##**LightGBM (Light Gradient Boosting Machine)**
**Funcionamento**
O LightGBM é uma implementação eficiente de Gradient Boosting desenvolvida pela Microsoft. Ele utiliza técnicas como histogram-based learning e leaf-wise growth para acelerar o treinamento e reduzir o consumo de memória, mantendo ou melhorando a precisão.

**Pontos Fortes**
- Rapidez: Treinamento mais rápido comparado ao XGBoost devido a otimizações internas.
- Baixo Consumo de Memória: Eficiente no uso de memória, permitindo trabalhar com grandes conjuntos de dados.
- Alto Desempenho: Similar ou superior ao XGBoost em muitas tarefas.
- Suporte a Dados Categóricos: Lida de forma nativa com features categóricas.

**Pontos Fracos**
- Sensível aos Hiperparâmetros: Requer ajuste cuidadoso para alcançar o melhor desempenho.
- Complexidade de Implementação: Pode ser mais complexo de implementar em algumas situações.
- Menos Estável em Algumas - Configurações: Pode apresentar variações de desempenho dependendo dos dados e parâmetros.

In [9]:
# 8. LightGBM
lgb_clf = lgb.LGBMClassifier(random_state=42)
train_evaluate_model(lgb_clf, "LightGBM", X_train, X_test, y_train, y_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 249, number of negative: 149
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000547 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3978
[LightGBM] [Info] Number of data points in the train set: 398, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.625628 -> initscore=0.513507
[LightGBM] [Info] Start training from score 0.513507
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 